# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Вывести все адреса электронной почты, содержащиеся в адресной книге `addres-book.json`

In [ ]:
import json

with open(r'data_f/addres-book.json') as f:
    data = json.load(f)

for pers in data:
    email = pers['email']
    print(email)
 

2. Вывести телефоны, содержащиеся в адресной книге `addres-book.json`

In [ ]:
numb = [phone_entry['phone'] for person in data for phone_entry in person.get('phones')]
numb

3. По данным из файла `addres-book-q.xml` сформировать список словарей с телефонами каждого из людей. 

In [ ]:
from bs4 import BeautifulSoup

with open(r'data_f/addres-book-q.xml', 'r') as f:
    data = f.read() 


bs_data = BeautifulSoup(data, 'xml')

In [ ]:
people_list = []

for person_elem in bs_data.find_all('address'):
    person_dict = {'name': person_elem.find('name').text, 'phones': []}

    # Iterate over each 'phone' element within the current 'person'
    for phone_elem in person_elem.find_all('phone'):
        person_dict['phones'].append(phone_elem.text)

    # Append the current person's dictionary to the list
    people_list.append(person_dict)


for person_dict in people_list:
    print(person_dict)

## Лабораторная работа №4

### JSON

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [ ]:
import json

with open(r'data_f/contributors_sample.json') as f:
    data = json.load(f)
    
data[:3]


1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [ ]:
l_of_domens = []
for i in data:
    ind = i['mail'].index('@') # находим индекс элемента
    domens = i['mail'][ind+1:] # достаем значения правее символа по индексу, не включая сам элемент
    l_of_domens.append(domens)

set(l_of_domens) # выводим уникальные значения

1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [ ]:
def search_username(nm):
    for i in data:
        if i['username'] == nm:
            return i
    raise ValueError()

try:
    print(search_username('uhebert'))
    print(search_username('uhebeet44yrt'))
except ValueError:
    print("Ошибка, Пользователя не существует")

1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [ ]:
count_f = 0
count_m = 0
for sex in data:
    if sex['sex'] == 'F':
        count_f += 1
    elif sex['sex'] == 'M':
        count_m += 1

print(f'Количество мужчин в наборе данных - {count_m} \nКоличество женщин в наборе данных - {count_f}')

1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [ ]:
import pandas as pd

selected_columns = ["id", "username", "sex"]

contributors = pd.DataFrame(data, columns=selected_columns)

contributors

1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [ ]:
import pandas as pd

recipes_file_path = r'data/recipes_sample.csv'
recipes = pd.read_csv(recipes_file_path)

recipes = pd.DataFrame(recipes)
    
merged_df = pd.merge(contributors, recipes, on='id', how='outer')

missing_contributors = merged_df[merged_df[['username']].isnull().any(axis=1)]

missing_count = len(missing_contributors)

# Выведем результат
print(f"Количество человек, информация о которых отсутствует: {missing_count}")

### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [ ]:
l_of_posts = {}

for i in data:
    jobs_list = i['jobs']
    pers = i['username']
    
    for job in jobs_list:
        if job not in l_of_posts:
            l_of_posts[job] = []
        l_of_posts[job].append(pers)
        
l_of_posts
        

2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [ ]:
import pickle
import json

with open('job_people.pickle', 'wb') as pickle_file:
    pickle.dump(l_of_posts, pickle_file)

with open('job_people.json', 'w') as json_file:
    json.dump(l_of_posts, json_file, indent = 3) 
 

2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [ ]:
with open('job_people.pickle', 'rb') as pickle_file:
    pk = pickle.load(pickle_file)
    
with open('job_people.json', 'r') as json_file:
    data_js = json.load(json_file)
    
pk

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [ ]:
from bs4 import BeautifulSoup

with open(r'data_f/steps_sample.xml', 'r') as f:
    data_xml = f.read() 



In [ ]:
bs_data_xml = BeautifulSoup(data_xml, 'xml')

d_of_steps = {}

for recipes_elem in bs_data_xml.find_all('recipes'):
    for recipe_elem in recipes_elem.find_all('recipe'):
        id_ = recipe_elem.find('id').text
        d_of_steps[id_] = []  # Initialize the list for each 'id'

        for step_elem in recipe_elem.find_all('steps'):
            # Assuming 'steps' contains multiple 'step' elements
            for step in step_elem.find_all('step'):
                d_of_steps[id_].append(step.text)
                
for key, value in list(d_of_steps.items())[:3]:
    print(f"ID: {key}, ШАГИ: {value}")

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

In [ ]:
d_of_ids = {}

for recipes_elem in bs_data_xml.find_all('recipes'):
    for recipe_elem in recipes_elem.find_all('recipe'):
        id_ = recipe_elem.find('id').text
        for step_elem in recipe_elem.find_all('steps'):
            l = len(step_elem)
            if l in d_of_ids:
                d_of_ids[l].append(id_)
            else:
                d_of_ids[l] = [id_] 
                
d_of_ids

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

In [ ]:
bs_data_xml.find_all('recipes')

In [ ]:
bs_data_xml.find_all('recipes')
# step has_minutes="1"    step has_hours="1"
lst = []
for recipe in bs_data_xml.find_all('recipe'):
    rec_id = recipe.find('id').text
    for step_elem in recipe.find_all('steps'):
        for step in step_elem.find_all('step'):
            if 'has_minutes' in step.attrs or 'has_hours' in step.attrs: # attrs - это 
                #атрибут объекта элемента BeautifulSoup, который представляет собой словарь, 
                #содержащий все атрибуты данного тега.
                lst.append(rec_id)
                break
                
len(lst)

3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

In [ ]:
import pandas as pd

recipes = r'data/recipes_sample.csv'
recipes = pd.read_csv(recipes)
print(recipes['n_steps'])

In [ ]:
steps_lengths = {}
for recipe in bs_data_xml.find_all('recipe'):
    recipe_id = int(recipe.find('id').text)
    steps = recipe.find('steps').find_all('step')
    steps_lengths[recipe_id] = len(steps)
        
for index, row in recipes.iterrows():
    recipe_id = row['id']
    if recipe_id in steps_lengths and pd.isna(row['n_steps']):
        recipes.at[index, 'n_steps'] = steps_lengths[recipe_id]

# Вывод результатов
print(recipes['n_steps'])


3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`

In [ ]:
missing_n_steps_indices = recipes['n_steps'].isna()
missing_n_steps_indices

In [ ]:
for elem in missing_n_steps_indices:
    if elem is False:
        recipes['n_steps'] = recipes['n_steps'].astype(int)
    else:
        print('Столбец содержит пропуски')

recipes.to_csv('recipes_sample_with_filled_nsteps.csv', index=False)